# Sybil exploration grant clima

In [1]:
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd

In [2]:
from legos import TransactionAnalyser
from utils import LoadData

In [3]:
current_dir = Path(os.getcwd())

### Initialisation of the data

#### Grant data initialisation

In [4]:
path_to_contributor_address = "data/grants/address"

In [5]:
full_path_add = os.path.join(current_dir, path_to_contributor_address)
files = os.listdir(full_path_add)
files

['df_contribution_address_CLIMATE.csv',
 'df_contribution_address_ETHEREUM.csv',
 'df_contribution_address_FANTOM.csv',
 'df_contribution_address_GR15.csv',
 'df_contribution_address_OSS.csv',
 'df_contribution_address_UNICEF.csv',
 'unique_ctbt_address.csv']

In [6]:
df_address = pd.read_csv(os.path.join(full_path_add, "df_contribution_address_CLIMATE.csv"))
list_address = df_address["address"].tolist()

The list_address is then used to load all the transactions from the grant data.

#### Data loader initialisation

In [7]:
path_to_parent = Path(current_dir).parent
path_to_tx = os.path.join(path_to_parent, 'transactions_full')
print(path_to_tx)

c:\Users\DELL Admin\Documents\ODC\DataBuilderHackathon\transactions_full


In [8]:
data_loader = LoadData.LoadData(path_to_tx)

The data loader is then used to load the data from the grant data. It takes some time to load the data. 3 minutes on my computer. 

In [9]:
df_tx = data_loader.create_df_tx('ethereum', list_address)

#### Sybil scorer TransactionAnalyser initialisation and exploration

In [10]:
tx_analyser = TransactionAnalyser.TransactionAnalyser(df_tx, df_address=df_address)

In [11]:
df_matching_address = pd.DataFrame(df_tx.EOA.unique(), columns=["address"])
df_matching_address.head(2)

,address
0,0x001f41de6f455d6df74a669c6b21b6dda1bae644
1,0x002153708f11f2651215059eea30820ee4d49ff3


In [19]:
'0x001f41de6f455d6df74a669c6b21b6dda1bae644' in tx_analyser.df_seed_wallet_naive.to_address.values

False

In [20]:
seed_add = tx_analyser.df_seed_wallet_naive.loc['0x001f41de6f455d6df74a669c6b21b6dda1bae644', 'from_address']

In [21]:
tx_analyser.df_seed_wallet_naive.loc['0x001f41de6f455d6df74a669c6b21b6dda1bae644', :]

from_address    0x001f41de6f455d6df74a669c6b21b6dda1bae644
to_address      0x1b165fe4da6bc58ab8370ddc763d367d29f50ef0
Name: 0x001f41de6f455d6df74a669c6b21b6dda1bae644, dtype: object

In [22]:
'0x001f41de6f455d6df74a669c6b21b6dda1bae644' in tx_analyser.df_seed_wallet_naive.index

True

In [50]:
df_matching_address['seed_same_naive'] = df_matching_address.loc[:, 'address'].apply(lambda x : tx_analyser.has_same_seed_naive(x))

In [51]:
df_matching_address['seed_same'] = df_matching_address.loc[:, 'address'].apply(lambda x : tx_analyser.has_same_seed(x))

In [52]:
df_matching_address['seed_suspicious'] = df_matching_address.loc[:, 'seed_same_naive'].ne(df_matching_address.loc[:, 'seed_same'])

In [53]:
df_matching_address['seed_suspicious'].value_counts()

False    2428
True       35
Name: seed_suspicious, dtype: int64

In [55]:
df_matching_address[df_matching_address['seed_suspicious'] == True]

,address,seed_same_naive,seed_same,seed_suspicious
14,0x0150c608a25176c08d9bc8b43d0dc64e4c9f4a58,False,True,True
57,0x061c405ccb27cf0ebfc76f41da09c2e3eba8c476,False,True,True
189,0x13969a736a633aab08b2d57881b0f4ec92ef9a86,False,True,True
266,0x1c491e2253abc092acadc5744a0623229e95b8a5,False,True,True
473,0x30b3bbdb92eeb77f0c3394bfaf26a44767ec1bd0,False,True,True
478,0x312e80bb497f5a399e01208c47360b531e34910b,False,True,True
498,0x32ff90bc3fa85d87e982d5b384a453bed65a799d,False,True,True
624,0x41b0b75802b25fbc14e0157f9341473843c28e1e,False,True,True
698,0x49371938108e13d0253de8f488de7c82c2735ee0,False,True,True
703,0x49f86794fe566dead8f8198729e5fdf37cb7f6ab,False,True,True


In [56]:
df_address['transaction_similitude'] = df_address['address'].apply(lambda x : tx_analyser.has_transaction_similitude_opti(x))


C:\Users\DELL Admin\Documents\ODC\DataBuilderHackathon\sybil-scorer\sbscorer\legos\TransactionAnalyser.py:274: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_other_address['lcs'] = 0
C:\Users\DELL Admin\Documents\ODC\DataBuilderHackathon\sybil-scorer\sbscorer\legos\TransactionAnalyser.py:287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_other_address.loc[add, 'lcs'] = lcs
C:\Users\DELL Admin\Documents\ODC\DataBuilderHackathon\sybil-scorer\sbscorer\legos\TransactionAnalyser.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [ ]:
tx_analyser = TransactionAnalyser()

TypeError: 'module' object is not callable